In [1]:
# merge data
# data transformation
# store preprocessing data
import numpy as np
import pandas as pd
import os

In [175]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
!dir data

 驱动器 C 中的卷没有标签。
 卷的序列号是 0006-FDC5

 C:\Users\dell\dockerdata\乘用车细分市场销量预测\data 的目录

2019/09/20  16:14    <DIR>          .
2019/09/20  16:14    <DIR>          ..
2019/09/11  13:30           237,991 evaluation_public.csv
2019/09/19  14:39            51,762 mean_predicting_data.csv
2019/09/11  13:31            51,712 submit_example.csv
2019/09/20  16:14            51,711 svr_predicting_sales.csv
2019/08/23  17:36           369,997 Train.zip
2019/09/12  15:33         1,545,277 train_sales_data.csv
2019/09/12  15:33         1,399,503 train_search_data.csv
2019/09/12  15:33            47,975 train_user_reply_data.csv
               8 个文件      3,755,928 字节
               2 个目录 47,160,934,400 可用字节


In [3]:
train_sales_data=pd.read_csv('data/train_sales_data.csv')

In [4]:
train_sales_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31680 entries, 0 to 31679
Data columns (total 7 columns):
province       31680 non-null object
adcode         31680 non-null int64
model          31680 non-null object
bodyType       31680 non-null object
regYear        31680 non-null int64
regMonth       31680 non-null int64
salesVolume    31680 non-null int64
dtypes: int64(4), object(3)
memory usage: 1.7+ MB


In [47]:
model_bodytype=train_sales_data[['model','bodyType']].drop_duplicates()
model_bodytype.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60 entries, 0 to 1298
Data columns (total 2 columns):
model       60 non-null object
bodyType    60 non-null object
dtypes: object(2)
memory usage: 1.4+ KB


In [5]:
evaluation_public_data=pd.read_csv('data/evaluation_public.csv')
evaluation_public_data.head()

,id,province,adcode,model,regYear,regMonth,forecastVolum
0,1,上海,310000,3c974920a76ac9c1,2018,1,NaN
1,2,云南,530000,3c974920a76ac9c1,2018,1,NaN
2,3,内蒙古,150000,3c974920a76ac9c1,2018,1,NaN
3,4,北京,110000,3c974920a76ac9c1,2018,1,NaN
4,5,四川,510000,3c974920a76ac9c1,2018,1,NaN


In [249]:
evaluation_public_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5280 entries, 0 to 5279
Data columns (total 8 columns):
id               5280 non-null int64
province         5280 non-null object
adcode           5280 non-null int64
model            5280 non-null object
regYear          5280 non-null int64
regMonth         5280 non-null int64
forecastVolum    0 non-null float64
date             5280 non-null datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(4), object(2)
memory usage: 330.1+ KB


In [6]:
train_sales_data_2017=train_sales_data[train_sales_data['regYear']==2017]
train_sales_data_2016=train_sales_data[train_sales_data['regYear']==2016]

In [7]:
train_sales_data_2017.head(1)

,province,adcode,model,bodyType,regYear,regMonth,salesVolume
15840,上海,310000,3c974920a76ac9c1,SUV,2017,1,305


In [8]:
train_sales_data_2017['regYear'].value_counts()

2017    15840
Name: regYear, dtype: int64

In [8]:
# merge data on the month of year before
def merge_year_before(df,spdf,cols):
    '''
    input:
        df(pd.DataFrame)-the left merging data df
        psdf(pd.DataFrame)-the right merging data df
    putput:
        mergeddf-the left merged data df
    '''
    onlist=['province','model','regMonth']
    mergeddf=pd.merge(df,spdf,on=onlist,how='left',suffixes=('','_year_before'))
    #cols=['province','adcode','model','bodyType','regYear','regMonth','salesVolume_year_before','salesVolume']
    return mergeddf[cols]

In [9]:
cols=['province','adcode','model','bodyType','regYear','regMonth','salesVolume_year_before','salesVolume']
train_sales_year_before_data=merge_year_before(train_sales_data_2017,train_sales_data_2016,cols)
train_sales_year_before_data.head()

,province,adcode,model,bodyType,regYear,regMonth,salesVolume_year_before,salesVolume
0,上海,310000,3c974920a76ac9c1,SUV,2017,1,292,305
1,云南,530000,3c974920a76ac9c1,SUV,2017,1,466,434
2,内蒙古,150000,3c974920a76ac9c1,SUV,2017,1,257,180
3,北京,110000,3c974920a76ac9c1,SUV,2017,1,408,468
4,四川,510000,3c974920a76ac9c1,SUV,2017,1,610,500


In [36]:
cols=['id','province','adcode','model','bodyType','regYear','regMonth','salesVolume']
evaluation_public_year_before_data=merge_year_before(evaluation_public_data,train_sales_data_2017,cols)
evaluation_public_year_before_data.columns=['id','province','adcode','model','bodyType','regYear',\
                                            'regMonth','salesVolume_year_before']
evaluation_public_year_before_data.head()

,id,province,adcode,model,bodyType,regYear,regMonth,salesVolume_year_before
0,1,上海,310000,3c974920a76ac9c1,SUV,2018,1,305
1,2,云南,530000,3c974920a76ac9c1,SUV,2018,1,434
2,3,内蒙古,150000,3c974920a76ac9c1,SUV,2018,1,180
3,4,北京,110000,3c974920a76ac9c1,SUV,2018,1,468
4,5,四川,510000,3c974920a76ac9c1,SUV,2018,1,500


In [11]:
# 生成 datetime 列
def generate_date(df):
    f=lambda x:str(x) if x >9 else '0'+str(x)
    df['date']=df['regYear'].map(lambda x:str(x))+'-'+df['regMonth'].map(f)
    df['date']=pd.to_datetime(df['date'],format='%Y-%m')
    return df

In [13]:
# 日期偏移 month
def month_offset_date(df,n=1):
    from pandas.tseries.offsets import DateOffset
    df['offdate']=df['date']+DateOffset(months=n)
    return df

In [14]:
# 日期偏移 year
def year_offset_date(df,n=1):
    from pandas.tseries.offsets import DateOffset
    df['offdate']=df['date']+DateOffset(years=n)
    return df

In [15]:
#merge data month before
def merge_month_before(df,spdf,cols):
    '''
    input:
        df(pd.DataFrame)--left merging data df
        spdf(pd.DataFrame)--right merging data df
    output:
        mergedf(pd.DataFrame)--merged data df
    '''
    
    df=generate_date(df)
    spdf=generate_date(spdf)
    spdf=month_offset_date(spdf)
    mergedf=pd.merge(df,spdf,how='left',left_on=['province','model','date'],right_on=['province','model','offdate'],\
                    suffixes=('','_month_before'))
    #cols=['id','province','adcode','model','regYear','regMonth','salesVolume_month_before']
    #return mergedf
    return mergedf[cols]

In [202]:
cols=['province','adcode','model','bodyType','regYear','regMonth','salesVolume_month_before']
train_sales_month_before=merge_month_before(train_sales_data_2017,train_sales_data,cols)
train_sales_month_before.head(2)

,province,adcode,model,bodyType,regYear,regMonth,salesVolume_month_before
0,上海,310000,3c974920a76ac9c1,SUV,2017,1,374
1,云南,530000,3c974920a76ac9c1,SUV,2017,1,456


In [60]:
train_sales_month_before.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15840 entries, 0 to 15839
Data columns (total 7 columns):
province                    15840 non-null object
adcode                      15840 non-null int64
model                       15840 non-null object
bodyType                    15840 non-null object
regYear                     15840 non-null int64
regMonth                    15840 non-null int64
salesVolume_month_before    15840 non-null int64
dtypes: int64(4), object(3)
memory usage: 990.0+ KB


In [61]:
train_data=pd.merge(train_sales_month_before,train_sales_year_before_data,\
                    on=['province','adcode','model','bodyType','regYear','regMonth'],\
                   how='left',suffixes=('','_right'))
train_data.head()

,province,adcode,model,bodyType,regYear,regMonth,salesVolume_month_before,salesVolume_year_before,salesVolume
0,上海,310000,3c974920a76ac9c1,SUV,2017,1,374,292,305
1,云南,530000,3c974920a76ac9c1,SUV,2017,1,456,466,434
2,内蒙古,150000,3c974920a76ac9c1,SUV,2017,1,219,257,180
3,北京,110000,3c974920a76ac9c1,SUV,2017,1,563,408,468
4,四川,510000,3c974920a76ac9c1,SUV,2017,1,570,610,500


In [63]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15840 entries, 0 to 15839
Data columns (total 9 columns):
province                    15840 non-null object
adcode                      15840 non-null int64
model                       15840 non-null object
bodyType                    15840 non-null object
regYear                     15840 non-null int64
regMonth                    15840 non-null int64
salesVolume_month_before    15840 non-null int64
salesVolume_year_before     15840 non-null int64
salesVolume                 15840 non-null int64
dtypes: int64(6), object(3)
memory usage: 1.2+ MB


In [164]:
test_cols=['id','province','adcode','model','bodyType','regYear','regMonth','salesVolume']
evaluation_public_month_data=merge_month_before(evaluation_public_data,train_sales_data_2017,test_cols)
evaluation_public_month_data.columns=['id','province','adcode','model','bodyType','regYear',\
                                            'regMonth','salesVolume_month_before']
evaluation_public_month_data.head()

,id,province,adcode,model,bodyType,regYear,regMonth,salesVolume_month_before
0,1,上海,310000,3c974920a76ac9c1,SUV,2018,1,312.0
1,2,云南,530000,3c974920a76ac9c1,SUV,2018,1,331.0
2,3,内蒙古,150000,3c974920a76ac9c1,SUV,2018,1,173.0
3,4,北京,110000,3c974920a76ac9c1,SUV,2018,1,275.0
4,5,四川,510000,3c974920a76ac9c1,SUV,2018,1,456.0


In [53]:
test_data=pd.merge(evaluation_public_month_data,evaluation_public_year_before_data,how='left',\
                   on=['id','province','adcode','model','regYear','regMonth'],suffixes=('','_right'))
test_data=pd.merge(test_data,model_bodytype,on='model',how='left',suffixes=('_left',''))
states=['id','province','adcode','model','bodyType','regYear','regMonth','salesVolume_month_before','salesVolume_year_before']
test_data=test_data.reindex(columns=states)
test_data.head()


,id,province,adcode,model,bodyType,regYear,regMonth,salesVolume_month_before,salesVolume_year_before
0,1,上海,310000,3c974920a76ac9c1,SUV,2018,1,312.0,305
1,2,云南,530000,3c974920a76ac9c1,SUV,2018,1,331.0,434
2,3,内蒙古,150000,3c974920a76ac9c1,SUV,2018,1,173.0,180
3,4,北京,110000,3c974920a76ac9c1,SUV,2018,1,275.0,468
4,5,四川,510000,3c974920a76ac9c1,SUV,2018,1,456.0,500


In [200]:
lsindex=(train_data['regMonth']==1)&(train_data['province']=='上海')&(train_data['model']=='3c974920a76ac9c1')
train_data[lsindex]

,province,adcode,model,bodyType,regYear,regMonth,salesVolume_month_before,salesVolume_year_before,salesVolume
0,上海,310000,3c974920a76ac9c1,SUV,2017,1,374,292,305


In [201]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5280 entries, 0 to 5279
Data columns (total 10 columns):
id                          5280 non-null int64
province                    5280 non-null object
adcode                      5280 non-null int64
model                       5280 non-null object
bodyType                    5280 non-null object
regYear                     5280 non-null int64
regMonth                    5280 non-null int64
salesVolume_month_before    1320 non-null float64
salesVolume_year_before     5280 non-null int64
date                        5280 non-null datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(5), object(3)
memory usage: 453.8+ KB


In [134]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.utils import shuffle
ohe=OneHotEncoder()
shuffle(train_data)
train_cate_ohe=ohe.fit_transform(train_data[['province','model','bodyType']])
train_x=np.concatenate((train_cate_ohe.toarray(),train_data[['regMonth','salesVolume_month_before','salesVolume_year_before']].values),\
                 axis=1)
train_y=train_data['salesVolume'].values.ravel()

In [163]:
# Linear_model L1 Lasso
from sklearn import linear_model
from sklearn.model_selection import train_test_split
x,eva_x,y,eva_y=train_test_split(train_x,train_y,test_size=0.2)
reg=linear_model.Lasso(alpha=0.001)
reg.fit(x,y)
reg.score(x,y)

0.902765129815807

In [136]:
reg.score(eva_x,eva_y)

0.9084218024194696

In [180]:
# Linear_model L2 Ridge
ridge=linear_model.Ridge(alpha=1.0)
ridge.fit(x,y)
ridge.score(x,y)

0.9027647525735991

In [182]:
ridge.score(eva_x,eva_y)

0.9052303868539404

In [186]:
#SVR 奇差无比
from sklearn.svm import SVR
svm_reg=SVR()
svm_reg.fit(x,y)
svm_reg.score(x,y)

-0.086177011335687

In [192]:
#RF 随机森林
from sklearn.ensemble import RandomForestRegressor
rf_reg=RandomForestRegressor()
rf_reg.fit(x,y)
cross_val_score(rf_reg,x,y,cv=5)

array([0.9373449 , 0.93246862, 0.93338535, 0.91275773, 0.93395575])

In [193]:
rf_reg.score(eva_x,eva_y)

0.9292721962986755

In [ ]:
# test data

In [195]:
test_data_jan.head()

,id,province,adcode,model,bodyType,regYear,regMonth,salesVolume_month_before,salesVolume_year_before,salesVolume,date,offdate
0,1,上海,310000,3c974920a76ac9c1,SUV,2018,1,312.0,305,209.0,2018-01-01,2018-02-01
1,2,云南,530000,3c974920a76ac9c1,SUV,2018,1,331.0,434,304.0,2018-01-01,2018-02-01
2,3,内蒙古,150000,3c974920a76ac9c1,SUV,2018,1,173.0,180,110.0,2018-01-01,2018-02-01
3,4,北京,110000,3c974920a76ac9c1,SUV,2018,1,275.0,468,204.0,2018-01-01,2018-02-01
4,5,四川,510000,3c974920a76ac9c1,SUV,2018,1,456.0,500,375.0,2018-01-01,2018-02-01


In [210]:
y_pred=rf_reg.predict(x)
n_samples=y_pred.shape[0]
y_mean=np.mean(y)

In [211]:
1-((((y-y_pred)**2).sum()/n_samples)**0.5)/y_mean

0.8522303780811853

In [114]:
y_mean=np.mean(y)

In [115]:
y_mean

575.4069677817173

In [130]:
test_data.head()

,id,province,adcode,model,bodyType,regYear,regMonth,salesVolume_month_before,salesVolume_year_before
0,1,上海,310000,3c974920a76ac9c1,SUV,2018,1,312.0,305
1,2,云南,530000,3c974920a76ac9c1,SUV,2018,1,331.0,434
2,3,内蒙古,150000,3c974920a76ac9c1,SUV,2018,1,173.0,180
3,4,北京,110000,3c974920a76ac9c1,SUV,2018,1,275.0,468
4,5,四川,510000,3c974920a76ac9c1,SUV,2018,1,456.0,500


In [131]:
#JAN
test_data_jan=test_data[test_data['regMonth']==1]
test_x_jan=np.concatenate((ohe.transform(test_data_jan[['province','model','bodyType']].values).toarray(),\
                           test_data_jan[['regMonth','salesVolume_month_before','salesVolume_year_before']]),axis=1)

In [207]:
test_y_pred_jan=rf_reg.predict(test_x_jan)
test_data_jan['salesVolume']=np.rint(test_y_pred_jan)

In [208]:
test_data_jan.head(2)

,id,province,adcode,model,bodyType,regYear,regMonth,salesVolume_month_before,salesVolume_year_before,salesVolume,date,offdate
0,1,上海,310000,3c974920a76ac9c1,SUV,2018,1,312.0,305,250.0,2018-01-01,2018-02-01
1,2,云南,530000,3c974920a76ac9c1,SUV,2018,1,331.0,434,282.0,2018-01-01,2018-02-01


In [229]:
#Febrary
test_cols=['id','province','adcode','model','bodyType','regYear','regMonth','salesVolume']
test_cols.append('salesVolume_year_before')
test_data_merge=merge_month_before(test_data,test_data_jan,test_cols)
test_data_merge.columns=['id','province','adcode','model','bodyType','regYear','regMonth','salesVolume_month_before','salesVolume_year_before']
test_data_merge[lsdf['regMonth']==2].head()

,id,province,adcode,model,bodyType,regYear,regMonth,salesVolume_month_before,salesVolume_year_before
1320,1343,上海,310000,3c974920a76ac9c1,SUV,2018,2,250.0,193
1321,1344,云南,530000,3c974920a76ac9c1,SUV,2018,2,282.0,194
1322,1345,内蒙古,150000,3c974920a76ac9c1,SUV,2018,2,114.0,128
1323,1346,北京,110000,3c974920a76ac9c1,SUV,2018,2,206.0,432
1324,1347,四川,510000,3c974920a76ac9c1,SUV,2018,2,486.0,253


In [230]:

test_data_feb=test_data_merge[test_data_merge['regMonth']==2]
test_x_feb=np.concatenate((ohe.transform(test_data_feb[['province','model','bodyType']].values).toarray(),\
                           test_data_feb[['regMonth','salesVolume_month_before','salesVolume_year_before']]),axis=1)
test_y_pred_feb=rf_reg.predict(test_x_feb)
test_data_feb['salesVolume']=np.rint(test_y_pred_feb)
test_data_feb.head()

,id,province,adcode,model,bodyType,regYear,regMonth,salesVolume_month_before,salesVolume_year_before,salesVolume
1320,1343,上海,310000,3c974920a76ac9c1,SUV,2018,2,250.0,193,123.0
1321,1344,云南,530000,3c974920a76ac9c1,SUV,2018,2,282.0,194,161.0
1322,1345,内蒙古,150000,3c974920a76ac9c1,SUV,2018,2,114.0,128,106.0
1323,1346,北京,110000,3c974920a76ac9c1,SUV,2018,2,206.0,432,218.0
1324,1347,四川,510000,3c974920a76ac9c1,SUV,2018,2,486.0,253,240.0


In [232]:
#March
test_cols=['id','province','adcode','model','bodyType','regYear','regMonth','salesVolume']
test_cols.append('salesVolume_year_before')
test_data_merge=merge_month_before(test_data_merge,test_data_feb,test_cols)
test_data_merge.columns=['id','province','adcode','model','bodyType','regYear','regMonth','salesVolume_month_before','salesVolume_year_before']
test_data_merge[lsdf['regMonth']==3].head()

,id,province,adcode,model,bodyType,regYear,regMonth,salesVolume_month_before,salesVolume_year_before
2640,2685,上海,310000,3c974920a76ac9c1,SUV,2018,3,123.0,309
2641,2686,云南,530000,3c974920a76ac9c1,SUV,2018,3,161.0,302
2642,2687,内蒙古,150000,3c974920a76ac9c1,SUV,2018,3,106.0,201
2643,2688,北京,110000,3c974920a76ac9c1,SUV,2018,3,218.0,472
2644,2689,四川,510000,3c974920a76ac9c1,SUV,2018,3,240.0,378


In [233]:
test_data_mar=test_data_merge[test_data_merge['regMonth']==3]
test_x_mar=np.concatenate((ohe.transform(test_data_mar[['province','model','bodyType']].values).toarray(),\
                           test_data_mar[['regMonth','salesVolume_month_before','salesVolume_year_before']]),axis=1)
test_y_pred_mar=rf_reg.predict(test_x_mar)
test_data_mar['salesVolume']=np.rint(test_y_pred_mar)
test_data_mar.head()

,id,province,adcode,model,bodyType,regYear,regMonth,salesVolume_month_before,salesVolume_year_before,salesVolume
2640,2685,上海,310000,3c974920a76ac9c1,SUV,2018,3,123.0,309,228.0
2641,2686,云南,530000,3c974920a76ac9c1,SUV,2018,3,161.0,302,248.0
2642,2687,内蒙古,150000,3c974920a76ac9c1,SUV,2018,3,106.0,201,161.0
2643,2688,北京,110000,3c974920a76ac9c1,SUV,2018,3,218.0,472,328.0
2644,2689,四川,510000,3c974920a76ac9c1,SUV,2018,3,240.0,378,373.0


In [234]:
#April
test_cols=['id','province','adcode','model','bodyType','regYear','regMonth','salesVolume']
test_cols.append('salesVolume_year_before')
test_data_merge=merge_month_before(test_data_merge,test_data_mar,test_cols)
test_data_merge.columns=['id','province','adcode','model','bodyType','regYear','regMonth','salesVolume_month_before','salesVolume_year_before']
test_data_merge[lsdf['regMonth']==4].head()

,id,province,adcode,model,bodyType,regYear,regMonth,salesVolume_month_before,salesVolume_year_before
3960,4027,上海,310000,3c974920a76ac9c1,SUV,2018,4,228.0,264
3961,4028,云南,530000,3c974920a76ac9c1,SUV,2018,4,248.0,243
3962,4029,内蒙古,150000,3c974920a76ac9c1,SUV,2018,4,161.0,176
3963,4030,北京,110000,3c974920a76ac9c1,SUV,2018,4,328.0,340
3964,4031,四川,510000,3c974920a76ac9c1,SUV,2018,4,373.0,337


In [235]:
test_data_apr=test_data_merge[test_data_merge['regMonth']==4]
test_x_apr=np.concatenate((ohe.transform(test_data_apr[['province','model','bodyType']].values).toarray(),\
                           test_data_apr[['regMonth','salesVolume_month_before','salesVolume_year_before']]),axis=1)
test_y_pred_apr=rf_reg.predict(test_x_apr)
test_data_apr['salesVolume']=np.rint(test_y_pred_apr)
test_data_apr.head()

,id,province,adcode,model,bodyType,regYear,regMonth,salesVolume_month_before,salesVolume_year_before,salesVolume
3960,4027,上海,310000,3c974920a76ac9c1,SUV,2018,4,228.0,264,247.0
3961,4028,云南,530000,3c974920a76ac9c1,SUV,2018,4,248.0,243,228.0
3962,4029,内蒙古,150000,3c974920a76ac9c1,SUV,2018,4,161.0,176,160.0
3963,4030,北京,110000,3c974920a76ac9c1,SUV,2018,4,328.0,340,279.0
3964,4031,四川,510000,3c974920a76ac9c1,SUV,2018,4,373.0,337,381.0


In [245]:
test_data_pred=pd.concat([test_data_jan,test_data_feb,test_data_mar,test_data_apr])

In [246]:
test_data_apr.tail()

,id,province,adcode,model,bodyType,regYear,regMonth,salesVolume_month_before,salesVolume_year_before,salesVolume
5275,5364,福建,350000,a9a43d1a7ecbe75d,SUV,2018,4,93.0,88,87.0
5276,5365,辽宁,210000,a9a43d1a7ecbe75d,SUV,2018,4,88.0,102,88.0
5277,5366,重庆,500000,a9a43d1a7ecbe75d,SUV,2018,4,91.0,134,111.0
5278,5367,陕西,610000,a9a43d1a7ecbe75d,SUV,2018,4,266.0,212,219.0
5279,5368,黑龙江,230000,a9a43d1a7ecbe75d,SUV,2018,4,52.0,67,64.0


In [247]:
test_data_pred.tail()

,adcode,bodyType,date,id,model,offdate,province,regMonth,regYear,salesVolume,salesVolume_month_before,salesVolume_year_before
5275,350000,SUV,NaT,5364,a9a43d1a7ecbe75d,NaT,福建,4,2018,87.0,93.0,88
5276,210000,SUV,NaT,5365,a9a43d1a7ecbe75d,NaT,辽宁,4,2018,88.0,88.0,102
5277,500000,SUV,NaT,5366,a9a43d1a7ecbe75d,NaT,重庆,4,2018,111.0,91.0,134
5278,610000,SUV,NaT,5367,a9a43d1a7ecbe75d,NaT,陕西,4,2018,219.0,266.0,212
5279,230000,SUV,NaT,5368,a9a43d1a7ecbe75d,NaT,黑龙江,4,2018,64.0,52.0,67


In [248]:
test_data_pred['forecastVolum']=test_data_pred['salesVolume'].map(lambda x:int(x))
test_data_pred[['id','forecastVolum']].to_csv('data/rf_prediction_default.csv',index=False)